In [2]:
import pandas as pd
import tensorflow as tf

In [3]:
data = pd.read_csv("/kaggle/input/early-detection-of-3d-printing-issues/train.csv")

In [4]:
data.head()

,img_path,printer_id,print_id,has_under_extrusion
0,101/1678589738/1678589914.060332.jpg,101,1678589738,1
1,101/1678589738/1678589914.462857.jpg,101,1678589738,1
2,101/1678589738/1678589914.875075.jpg,101,1678589738,1
3,101/1678589738/1678589915.284495.jpg,101,1678589738,1
4,101/1678589738/1678589915.700486.jpg,101,1678589738,1


In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 81060 entries, 0 to 81059
Data columns (total 4 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   img_path             81060 non-null  object
 1   printer_id           81060 non-null  int64 
 2   print_id             81060 non-null  int64 
 3   has_under_extrusion  81060 non-null  int64 
dtypes: int64(3), object(1)
memory usage: 2.5+ MB


In [6]:
data["img_path"]= ["/kaggle/input/early-detection-of-3d-printing-issues/images/" + each for each in data["img_path"]]

In [7]:
data = data.sample(frac = 1)
train, val = data[0:int(0.8*len(data))], data[int(0.8*len(data)):]

In [8]:
train.head()

,img_path,printer_id,print_id,has_under_extrusion
47333,/kaggle/input/early-detection-of-3d-printing-i...,103,1679009928,1
65370,/kaggle/input/early-detection-of-3d-printing-i...,104,1678414274,0
14561,/kaggle/input/early-detection-of-3d-printing-i...,101,1678341667,0
60308,/kaggle/input/early-detection-of-3d-printing-i...,104,1679270824,1
6671,/kaggle/input/early-detection-of-3d-printing-i...,101,1678593348,1


In [9]:
val.head()

,img_path,printer_id,print_id,has_under_extrusion
50175,/kaggle/input/early-detection-of-3d-printing-i...,103,1678832038,0
60425,/kaggle/input/early-detection-of-3d-printing-i...,104,1679270824,1
43945,/kaggle/input/early-detection-of-3d-printing-i...,103,1679007451,1
54656,/kaggle/input/early-detection-of-3d-printing-i...,104,1678453862,1
46225,/kaggle/input/early-detection-of-3d-printing-i...,103,1679009928,1


In [10]:
temp = tf.io.read_file(data["img_path"][250])
temp = tf.image.decode_png(temp, channels=3)
print(temp.shape)

(480, 640, 3)


In [11]:
train_x = train["img_path"]
train_y = train["has_under_extrusion"]

val_x = val["img_path"]
val_y = val["has_under_extrusion"]

def load(file_path):
    img = tf.io.read_file(file_path)
    img = tf.image.decode_png(img, channels=3)
    img = tf.image.convert_image_dtype(img, tf.float32)
    img = tf.image.resize(img, size=(224, 224))
    return img

def one_hot(labels):
    return tf.one_hot(labels,depth=2)

train_ds = tf.data.Dataset.from_tensor_slices((train_x, train_y)).map(lambda xx, yy: (load(xx), one_hot(yy))).shuffle(buffer_size=1000).batch(32)
val_ds = tf.data.Dataset.from_tensor_slices((val_x, val_y)).map(lambda xx, yy: (load(xx), one_hot(yy))).shuffle(buffer_size=1000).batch(32)


In [ ]:
next(iter(train_ds))

In [12]:
#resnet
model = tf.keras.applications.resnet_v2.ResNet101V2(
    include_top=False,
    weights="imagenet",
    input_tensor=None,
    input_shape=(224,224,3),
    pooling=None,
    classes=1000,
    classifier_activation="sigmoid"
)


inputs = tf.keras.Input(shape=(224, 224, 3))
x = model(inputs, training=True)
x = tf.keras.layers.Flatten()(x)
outputs = tf.keras.layers.Dense(2,activation='softmax')(x)
model = tf.keras.Model(inputs, outputs)

171317808/171317808 [==============================] - 1s 0us/step


In [ ]:
model = tf.keras.applications.inception_resnet_v2.InceptionResNetV2(
    include_top=False,
    weights="imagenet",
    input_tensor=None,
    input_shape=(224,224,3),
    pooling=None,
    classes=1000,
    classifier_activation="sigmoid"
)

inputs = tf.keras.Input(shape=(224, 224, 3))
x = model(inputs, training=True)
x = tf.keras.layers.Flatten()(x)
outputs = tf.keras.layers.Dense(2, activation='softmax')(x)
model = tf.keras.Model(inputs, outputs)


In [13]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 resnet101v2 (Functional)    (None, 7, 7, 2048)        42626560  
                                                                 
 flatten (Flatten)           (None, 100352)            0         
                                                                 
 dense (Dense)               (None, 2)                 200706    
                                                                 
Total params: 42,827,266
Trainable params: 42,729,602
Non-trainable params: 97,664
_________________________________________________________________


In [14]:
optimizer = tf.keras.optimizers.Adam(learning_rate = 0.01)
model.compile(optimizer=optimizer,loss=tf.keras.losses.binary_crossentropy, metrics = ["accuracy"])

In [ ]:
history = model.fit(train_ds,epochs = 50,batch_size=4,validation_data = val_ds)

Epoch 1/50
2027/2027 [==============================] - 1239s 575ms/step - loss: 2.0631 - accuracy: 0.6486 - val_loss: 1.3493 - val_accuracy: 0.8583
Epoch 2/50
2027/2027 [==============================] - 1214s 596ms/step - loss: 5.2201 - accuracy: 0.8694 - val_loss: 1.7266 - val_accuracy: 0.9020
Epoch 3/50
2027/2027 [==============================] - 1212s 595ms/step - loss: 0.9799 - accuracy: 0.9173 - val_loss: 0.4762 - val_accuracy: 0.9351
Epoch 4/50
2027/2027 [==============================] - 1171s 574ms/step - loss: 0.7160 - accuracy: 0.9418 - val_loss: 0.0627 - val_accuracy: 0.9798
Epoch 5/50
2027/2027 [==============================] - 1217s 597ms/step - loss: 0.0577 - accuracy: 0.9800 - val_loss: 0.0461 - val_accuracy: 0.9841
Epoch 6/50
2027/2027 [==============================] - 1172s 575ms/step - loss: 0.0404 - accuracy: 0.9858 - val_loss: 0.0311 - val_accuracy: 0.9893
Epoch 7/50
2027/2027 [==============================] - 1217s 597ms/step - loss: 0.0292 - accuracy: 0.9899

NameError: name 'python' is not defined